## 経済産業省の通商白書を全文検索するAPIサーバの計画

Date: 2024/04/29-2024/04/30

注意：最終的には、汎用化し、経済産業省以外の省庁が発行する白書も検索可能にする予定。

## 前処理
- Beautiful SoapでPDFファイルのリンクリスト作成
- PyMuPDFで各PDFファイルのText抽出しSQLiteへ格納
- 同時にspaCyでNERを行い、その結果をSQLiteへ格納

```
Table "source" <= 経産省、総務省、XX株式会社といった白書やIR資料を提供している団体名
base_url(primary key), org

Table "links" <= Beautiful Soapで抽出したPDFファイルのURl
path(primary key), title, fk(source:base_url)

Table "texts" <= PyMuPDFで抽出したテキスト
fk(links:path), page, text

Table "named_entities" <= spaCyで抽出したNamed Entity
fk(texts:path), page, named_entity, label
```

## APIサーバ (Flaskで実装)

#### /sources

\[{base_url: \<base_url\>, org: \<base_url\>},...\]

PDFファイル検索対象となる団体のリストを返信（今回は経済産業省のみ）。

#### /search?base_url=\<base_url\>&keywords=\<keywords\>

\[{base_url: \<base_url\>, path: \<path\>, pages: \<pages\>},...\]

該当キーワードを含むPDFファイルのパスとページ番号を返信。

#### /highlight?base_url=\<base_url\>&path=\<path\>&page=\<page\>&keywords=\<keywords\>

該当キーワードをハイライトしてPDFを返送。keywordsなしのときはハイライトせず返信。指定ページの前後で合計３ページ分を返信。

## Table: source

In [1]:
import sqlite3

base_url_meti = "https://www.meti.go.jp"
org_meti = "経済産業省"
doc = "通商白書"

DB_PATH = 'database/search.db'

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS source')
    cur.execute('CREATE TABLE source (base_url TEXT PRIMARY KEY, org TEXT, doc TEXT, UNIQUE(org, doc))')
    cur.execute(f'INSERT INTO source (base_url, org, doc) VALUES ("{base_url_meti}", "{org_meti}", "{doc}")')

## Table: links

PDFリンク抽出

In [2]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    base_url = cur.execute('SELECT base_url FROM source WHERE org = "経済産業省" AND doc="通商白書"').fetchone()[0]

base_url

'https://www.meti.go.jp'

In [3]:
url_meti = "https://www.meti.go.jp/report/tsuhaku2023/whitepaper_2023.html"

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
resp = requests.get(url_meti)
html_doc = resp.content.decode('utf-8')

In [6]:
soup = BeautifulSoup(html_doc, 'html.parser')

In [7]:
chapter1 = [tag for tag in soup.find_all(string='第Ⅰ部　岐路に立たされる世界経済')]

In [8]:
chapter1[0].find_all_next("ul", {"class": "lnkLst"})

[<ul class="lnkLst">
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-1.pdf">第1節　世界経済の現状と見通し（PDF形式：3,448KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-2.pdf">第2節　ロシアによるウクライナ侵略を巡る状況とその影響（PDF形式：4,379KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-3.pdf">第3節　高まるインフレ圧力（PDF形式：4,415KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-4.pdf">第4節　新興国・途上国で高まる債務リスク（PDF形式：4,771KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-c1.pdf">コラム1　1980年代の中南米の累積債務問題（PDF形式：1,776KB）<img alt="PDFファイル" height="16" src="/img_2017/template/icon_pdf.gif" width="16"/></a></li>
 <li>
 <a href="/report/tsuhaku2023/pdf/1-1-5.pdf">第5節　分断の危機に直面する世界経済（PDF形式：3,758KB）<img alt="PD

In [9]:
all_ul = chapter1[0].find_all_next("ul", {"class": "lnkLst"})

all_as = []
for ul in all_ul:
    #print(ul.find_all('a'))
    all_as.extend(ul.find_all('a'))

# [[url, title], ...]
links = filter(lambda x: x[0].endswith('.pdf') ,[[a['href'], a.text] for a in all_as])

In [10]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS links')
    cur.execute('CREATE TABLE links (path TEXT PRIMARY KEY, title TEXT, base_url TEXT, FOREIGN KEY(base_url) REFERENCES source(base_url))')

    for path, title in links:
        cur.execute(f'INSERT INTO links (path, title, base_url) VALUES(?, ?, ?)', (path, title, base_url))

## Table: texts

In [11]:
from urllib.parse import urljoin

with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    links = cur.execute('SELECT * FROM links').fetchall()
    
all_url = [urljoin(e[2], e[0]) for e in links]

In [12]:
import requests
import fitz

resp = requests.get(all_url[0])
doc = fitz.open(stream=resp.content)

In [13]:
extract_text = lambda page: (page.number, page.get_text("text").replace('\n', ''))

[extract_text(page) for page in doc]

[(0, '岐路に立たされる世界経済第Ⅰ部'),
 (1, ''),
 (2,
  '第１節 \x07世界経済の現状と見通し第２節 \x07ロシアによるウクライナ侵略を巡る状況とその影響第３節 \x07高まるインフレ圧力第４節 新興国・途上国で高まる債務リスク第５節 分断の危機に直面する世界経済第１章減速感を強める世界経済'),
 (3,
  '\u3000本章では、はじめに、世界経済は、ロシアによるウクライナ侵略による不確実性の高まりやインフレの高進、金融引締めの加速により減速感を強めていること、欧米を中心とした急速な金融引締めは、通貨価値の下落、金利上昇を通じてグローバル・サウスを中心に債務リスクを高めていることを示す。\u3000つぎに、権威主義国の台頭により、世界経済は今、分断の危機に直面する一方、グローバル・サウスは中立的立場で自国の利益を確保する構図となっていることを示す。\u3000最後に、近年、経済依存関係を武器化する経済的威圧に係る事案が増加しており、 WTO の上級委員会が機能不全に陥る中、欧米諸国では対応の検討を加速していることを示す。世界経済の現状と見通し第１節（1）2022 年の世界経済の概要\u30002022 年の世界経済は成長鈍化の1 年であった。世界経済の成長率は、2021 年にはパンデミックからの回復により大幅な上昇を見せたものの、2022 年には成長に落ち着きが見られた。成長鈍化の主な要因としては、ロシアによるウクライナ侵略、インフレの加速、中国経済の成長鈍化があげられる。ロシアによるウクライナ侵略は、天然ガスや原油といった資源価格等の高騰、サプライチェーンの断絶とそれに伴う波及的な効果を発生させ、世界経済の成長にとってマイナスに寄与した。インフレ率の高まりは、パンデミック後の需要回復に伴い2021 年の後半から見られ、ロシアによるウクライナ侵略による資源価格の高騰や供給減少がインフレ率の高まりに拍車をかけた。インフレ率の高まりと、それに対処するための各国中央銀行の政策金利の急速な引上げは、家計の購買力の低下や経済活動の停滞、債務の持続可能性に影響を与え、世界経済の成長鈍化の要因となっている。中国経済は、パンデミックの再拡大と都市封鎖の影響により成長が鈍化し、それが世界経済の成長鈍化にも影響を与えた。\u30002022 年1 

In [14]:
# 本番処理
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    cur.execute('DROP TABLE IF EXISTS texts')
    cur.execute('CREATE TABLE texts (path TEXT, page INTEGER, text TEXT, FOREIGN KEY(path) REFERENCES links(path))')

    total = len(links)
    cnt = 1
    
    for link in links:
        print(f'{cnt}/{total}', end=' ')
        cnt += 1
        path = link[0]
        url = urljoin(link[2], link[0])
        resp = requests.get(url)
        doc = fitz.open(stream=resp.content)
        for page in doc:
            page_num, text = extract_text(page)
            cur.execute('INSERT INTO texts (path, page, text) VALUES (?, ?, ?)', (path, page_num, text))

1/41 2/41 3/41 4/41 5/41 6/41 7/41 8/41 9/41 10/41 11/41 12/41 13/41 14/41 15/41 16/41 17/41 18/41 19/41 20/41 21/41 22/41 23/41 24/41 25/41 26/41 27/41 28/41 29/41 30/41 31/41 32/41 33/41 34/41 35/41 36/41 37/41 38/41 39/41 40/41 41/41 

## Table: named_entities

In [15]:
import spacy

#!python3 -m spacy download ja_core_news_trf
nlp = spacy.load("ja_core_news_trf")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [16]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    texts = cur.execute('SELECT * FROM texts').fetchall()

In [17]:
texts[:3]

[('/report/tsuhaku2023/pdf/1-1-1.pdf', 0, '岐路に立たされる世界経済第Ⅰ部'),
 ('/report/tsuhaku2023/pdf/1-1-1.pdf', 1, ''),
 ('/report/tsuhaku2023/pdf/1-1-1.pdf',
  2,
  '第１節 \x07世界経済の現状と見通し第２節 \x07ロシアによるウクライナ侵略を巡る状況とその影響第３節 \x07高まるインフレ圧力第４節 新興国・途上国で高まる債務リスク第５節 分断の危機に直面する世界経済第１章減速感を強める世界経済')]

In [18]:
texts_ = [e[2] for e in texts]
texts_[:3]

['岐路に立たされる世界経済第Ⅰ部',
 '',
 '第１節 \x07世界経済の現状と見通し第２節 \x07ロシアによるウクライナ侵略を巡る状況とその影響第３節 \x07高まるインフレ圧力第４節 新興国・途上国で高まる債務リスク第５節 分断の危機に直面する世界経済第１章減速感を強める世界経済']

In [19]:
# 本番処理 Mac Book Air M1で１０分くらいかかる。
SKIP = True
if not SKIP:
    with sqlite3.connect(DB_PATH) as conn:
        cur = conn.cursor()
        cur.execute('DROP TABLE IF EXISTS named_entities')
        cur.execute('CREATE TABLE named_entities (path TEXT, page INTEGER, named_entity TEXT, label TEXT, \
        FOREIGN KEY(path) REFERENCES links(path))')
    
        total = len(texts)
        cnt = 1
        
        for text in texts:
            print(f'{cnt}/{total}', end=' ')
            cnt += 1
            
            path = text[0]
            page_num = text[1]
            text_ = text[2]
    
            doc = nlp(text_)
            for ent in doc.ents:
                named_entity = ent.text
                label = ent.label_
                
                #print(path, page_num, named_entity, label)
                cur.execute('INSERT INTO named_entities (path, page, named_entity, label) VALUES (?, ?, ?, ?)',
                            (path, page_num, named_entity, label))

## Named Entity のランキング

In [20]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    counts = cur.execute('SELECT named_entity, label, COUNT(named_entity) FROM named_entities \
    GROUP BY named_entity, label ORDER BY COUNT(named_entity) DESC').fetchall()

In [21]:
counts

[('日本', 'GPE', 461),
 ('中国', 'GPE', 392),
 ('ロシア', 'GPE', 324),
 ('米国', 'GPE', 322),
 ('ウクライナ', 'GPE', 218),
 ('2022 年', 'DATE', 211),
 ('\x08', 'NORP', 157),
 ('インド', 'GPE', 153),
 ('アジア', 'LOC', 130),
 ('経済産業省', 'ORG', 121),
 ('\x01', 'NORP', 120),
 ('EU', 'NORP', 117),
 ('2021 年', 'DATE', 114),
 ('欧州', 'LOC', 109),
 ('.', 'QUANTITY', 107),
 ('第１章', 'ORDINAL', 106),
 ('第２章', 'ORDINAL', 104),
 ('英国', 'GPE', 95),
 ('ドル', 'MONEY', 89),
 ('IMF', 'ORG', 86),
 ('経済産業大臣', 'TITLE_AFFIX', 86),
 ('CEIC database', 'ORG', 83),
 ('\x07', 'NORP', 82),
 ('西村', 'PERSON', 81),
 ('2', 'ORDINAL', 78),
 ('\x08\u2002', 'MOVEMENT', 75),
 ('中南米', 'LOC', 75),
 ('2023 年3 月', 'DATE', 72),
 ('WTO', 'ORG', 71),
 ('2019 年', 'DATE', 70),
 ('ASEAN', 'ORG', 70),
 ('インドネシア', 'GPE', 69),
 ('2023 年', 'DATE', 66),
 ('2020 年', 'DATE', 62),
 ('シンガポール', 'GPE', 58),
 ('2022 年度', 'DATE', 57),
 ('タイ', 'GPE', 57),
 ('韓国', 'GPE', 57),
 ('コロナ禍', 'EVENT', 56),
 ('1', 'ORDINAL', 54),
 ('2023 年1 月', 'DATE', 54),
 ('大臣', 'TITLE_AFF

In [22]:
[e for e in counts if e[1] == 'GPE']

[('日本', 'GPE', 461),
 ('中国', 'GPE', 392),
 ('ロシア', 'GPE', 324),
 ('米国', 'GPE', 322),
 ('ウクライナ', 'GPE', 218),
 ('インド', 'GPE', 153),
 ('英国', 'GPE', 95),
 ('インドネシア', 'GPE', 69),
 ('シンガポール', 'GPE', 58),
 ('タイ', 'GPE', 57),
 ('韓国', 'GPE', 57),
 ('ベトナム', 'GPE', 49),
 ('マレーシア', 'GPE', 46),
 ('ドイツ', 'GPE', 42),
 ('カナダ', 'GPE', 41),
 ('ブラジル', 'GPE', 39),
 ('メキシコ', 'GPE', 38),
 ('ASEAN', 'GPE', 37),
 ('フランス', 'GPE', 36),
 ('台湾', 'GPE', 31),
 ('フィリピン', 'GPE', 29),
 ('トルコ', 'GPE', 28),
 ('豪州', 'GPE', 27),
 ('ニュージーランド', 'GPE', 23),
 ('ペルー', 'GPE', 22),
 ('アルゼンチン', 'GPE', 21),
 ('スリランカ', 'GPE', 20),
 ('日', 'GPE', 19),
 ('カンボジア', 'GPE', 18),
 ('スイス', 'GPE', 17),
 ('ベラルーシ', 'GPE', 17),
 ('東京', 'GPE', 17),
 ('チリ', 'GPE', 16),
 ('ロシア連邦', 'GPE', 16),
 ('米中', 'GPE', 16),
 ('エジプト', 'GPE', 15),
 ('スペイン', 'GPE', 15),
 ('ケニア', 'GPE', 14),
 ('コロンビア', 'GPE', 14),
 ('ノルウェー', 'GPE', 14),
 ('パキスタン', 'GPE', 14),
 ('ガーナ', 'GPE', 13),
 ('ブルネイ', 'GPE', 13),
 ('アフリカ', 'GPE', 12),
 ('イタリア', 'GPE', 12),
 ('エチオピア', 'GPE',

## 特定ページの”ニュージーランド”をハイライトして出力

In [23]:
with sqlite3.connect(DB_PATH) as conn:
    cur = conn.cursor()
    new_zealand = cur.execute('SELECT * FROM named_entities WHERE named_entity = "ニュージーランド"').fetchall()

In [24]:
new_zealand

[('/report/tsuhaku2023/pdf/1-1-4.pdf', 0, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/1-2-3.pdf', 1, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/2-2-5.pdf', 9, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/2-2-5.pdf', 10, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/2-2-5.pdf', 11, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/2-2-5.pdf', 12, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-1.pdf', 9, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-3.pdf', 4, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-3.pdf', 4, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 1, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 1, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 1, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 3, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 3, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 3, 'ニュージーランド', 'GPE'),
 ('/report/tsuhaku2023/pdf/3-1-4.pdf', 4, 'ニュージーランド', 'GPE'),
 ('/r

In [25]:
sample = new_zealand[15]
sample

('/report/tsuhaku2023/pdf/3-1-4.pdf', 4, 'ニュージーランド', 'GPE')

In [26]:
path = sample[0]
page_num = sample[1]
named_entity = sample[2]
label = sample[3]
url = urljoin(base_url_meti, path)

In [27]:
import requests
import fitz
resp = requests.get(url)
doc = fitz.open(stream=resp.content)

In [28]:
doc[3].get_text()

'\u30002022 年6 月7 日に閣議決定された「経済財政運営\nと改革の基本方針 2022 新しい資本主義へ～課題解決\nを成長のエンジンに変え、持続可能な経済を実現～」\n（骨太方針2022）において、\n「多国間主義重視の下、\n人権を尊重し、環境にも配慮しつつ、自由で公正な経\n済圏の拡大、ルールに基づく多角的貿易体制の維持・\n強化に取り組む。\n（中略）\nTPP11 の着実な実施及び高\nいレベルを維持しながらの拡大に向けた議論を主導す\nるとともに、RCEP 協定の円滑な運用及び履行の確保\nに取り組む。IPEF については、インド太平洋地域へ\nの米国の強いコミットメントを示すものとして歓迎\nし、我が国は米国及びASEAN 諸国・インドを含む\nパートナー国と連携して地域の繁栄と経済秩序の構築\nに取り組み、加えて、米国にはTPP 復帰を働きかけ\nる。\n」と記載があるとおり、我が国はインド太平洋地\n域での協力等を通じ、経済連携をさらに推進し、自由\nで公正な貿易・投資ルールの実現をけん引する。\n（1）\x07\n環太平洋パートナーシップに関する包括的及\nび先進的な協定（CPTPP）\n（2018 年12 月30\n日発効）\n（a）CPTPP の概要\n\u3000我が国は、環太平洋パートナーシップ協定（以下、\nTPP 協定）に関し、2013 年3 月に参加を表明、同年\n7 月から豪州、ブルネイ、カナダ、チリ、マレーシア、\nメキシコ、\nニュージーランド、\nシンガポール、\nペルー、\n米国、ベトナムの11 か国との交渉に参加した。その\n後の交渉を経て、2015 年10 月に米国アトランタで大\n筋合意に至り、2016 年2 月4 日に署名がなされた。\n日本国内においては、2016 年12 月9 日に、TPP 協定\nが国会で承認されるとともに、関連法案が可決・成立\nした。その後、2017 年1 月20 日、TPP 協定原署名国\n12 か国の中で最も早く国内手続完了の通報を協定の\n寄託国であるニュージーランドに対して行った。\n\u3000一方、2017 年1 月30 日に米国がTPP からの離脱\nを参加各国に通告し、米国以外の11 か国の間で協定\nの早期発効を目指して協議が行われた。その結果、同\n年3 月

In [29]:
page = doc[3]
rects = page.search_for("ニュージーランド")
page.add_highlight_annot(rects)

'Highlight' annotation on page 3 of <memory, doc# 43>

In [32]:
doc2 = fitz.open()
doc2.insert_pdf(doc, from_page = 2, to_page = 4, start_at = 0)

In [33]:
doc2.save('tmp/highlighted_pages.pdf')